# Hyperparameter Optimization with Optuna

Пайплайн для автоматической оптимизации гиперпараметров:
- Bayesian optimization с Optuna
- Pruning неперспективных trials
- Multi-objective optimization
- Интеграция с CatBoost, XGBoost, LightGBM, Neural Networks

In [ ]:
!pip install optuna pandas numpy scikit-learn catboost xgboost lightgbm torch plotly -q

In [ ]:
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, f1_score
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

# Отключение логов Optuna (по желанию)
optuna.logging.set_verbosity(optuna.logging.WARNING)

print("✓ Библиотеки загружены!")

## 1. Загрузка данных

In [ ]:
# === ВАШИ ДАННЫЕ ===
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

TARGET_COL = 'target'
ID_COL = 'id'
TASK_TYPE = 'classification'  # 'classification' или 'regression'

# Подготовка данных
feature_cols = [col for col in train_df.columns if col not in [TARGET_COL, ID_COL]]
X = train_df[feature_cols]
y = train_df[TARGET_COL]
X_test = test_df[feature_cols]

# Train/Val split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, 
    stratify=y if TASK_TYPE == 'classification' else None
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

## 2. Optuna для XGBoost

In [ ]:
def objective_xgboost(trial):
    """
    Objective function для оптимизации XGBoost
    """
    # Определяем пространство гиперпараметров
    param = {
        'objective': 'binary:logistic' if TASK_TYPE == 'classification' else 'reg:squarederror',
        'eval_metric': 'auc' if TASK_TYPE == 'classification' else 'rmse',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
    }
    
    if param['booster'] in ['gbtree', 'dart']:
        param['max_depth'] = trial.suggest_int('max_depth', 3, 10)
        param['eta'] = trial.suggest_float('eta', 0.01, 0.3, log=True)
        param['gamma'] = trial.suggest_float('gamma', 1e-8, 1.0, log=True)
        param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
        param['min_child_weight'] = trial.suggest_int('min_child_weight', 1, 10)
        param['subsample'] = trial.suggest_float('subsample', 0.5, 1.0)
        param['colsample_bytree'] = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    
    if param['booster'] == 'dart':
        param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        param['rate_drop'] = trial.suggest_float('rate_drop', 1e-8, 1.0, log=True)
        param['skip_drop'] = trial.suggest_float('skip_drop', 1e-8, 1.0, log=True)
    
    # Cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) if TASK_TYPE == 'classification' else None
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    
    # Pruning callback
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'test-auc' if TASK_TYPE == 'classification' else 'test-rmse')
    
    cv_results = xgb.cv(
        param,
        dtrain,
        num_boost_round=1000,
        early_stopping_rounds=50,
        folds=cv,
        callbacks=[pruning_callback],
        verbose_eval=False
    )
    
    # Возвращаем лучший скор
    if TASK_TYPE == 'classification':
        return cv_results['test-auc-mean'].max()
    else:
        return cv_results['test-rmse-mean'].min()

# Создание study и оптимизация
study_xgb = optuna.create_study(
    direction='maximize' if TASK_TYPE == 'classification' else 'minimize',
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
    sampler=optuna.samplers.TPESampler(seed=42)
)

print("\nЗапуск оптимизации XGBoost...")
study_xgb.optimize(objective_xgboost, n_trials=50, timeout=600)  # 50 trials или 10 минут

print("\n✓ Оптимизация XGBoost завершена!")
print(f"Лучший скор: {study_xgb.best_value:.6f}")
print(f"Лучшие параметры: {study_xgb.best_params}")

## 3. Optuna для LightGBM

In [ ]:
def objective_lightgbm(trial):
    """
    Objective function для оптимизации LightGBM
    """
    param = {
        'objective': 'binary' if TASK_TYPE == 'classification' else 'regression',
        'metric': 'auc' if TASK_TYPE == 'classification' else 'rmse',
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'goss']),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0, 15),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
    }
    
    # Cross-validation с LightGBM
    dtrain = lgb.Dataset(X_train, label=y_train)
    
    # Pruning callback
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, 'auc' if TASK_TYPE == 'classification' else 'rmse')
    
    cv_results = lgb.cv(
        param,
        dtrain,
        num_boost_round=1000,
        nfold=5,
        stratified=TASK_TYPE == 'classification',
        callbacks=[pruning_callback, lgb.early_stopping(50)],
        return_cvbooster=False
    )
    
    if TASK_TYPE == 'classification':
        return max(cv_results['valid auc-mean'])
    else:
        return min(cv_results['valid rmse-mean'])

study_lgb = optuna.create_study(
    direction='maximize' if TASK_TYPE == 'classification' else 'minimize',
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
    sampler=optuna.samplers.TPESampler(seed=42)
)

print("\nЗапуск оптимизации LightGBM...")
study_lgb.optimize(objective_lightgbm, n_trials=50, timeout=600)

print("\n✓ Оптимизация LightGBM завершена!")
print(f"Лучший скор: {study_lgb.best_value:.6f}")
print(f"Лучшие параметры: {study_lgb.best_params}")

## 4. Optuna для CatBoost

In [ ]:
def objective_catboost(trial):
    """
    Objective function для оптимизации CatBoost
    """
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
        'random_strength': trial.suggest_float('random_strength', 1e-8, 10.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait': trial.suggest_int('od_wait', 10, 50),
        'random_state': 42,
        'verbose': 0
    }
    
    if param['bootstrap_type'] == 'Bayesian':
        param['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)
    elif param['bootstrap_type'] == 'Bernoulli':
        param['subsample'] = trial.suggest_float('subsample', 0.5, 1.0)
    
    # Определяем loss function
    if TASK_TYPE == 'classification':
        param['loss_function'] = 'Logloss'
        param['eval_metric'] = 'AUC'
    else:
        param['loss_function'] = 'RMSE'
        param['eval_metric'] = 'RMSE'
    
    # Обучение с валидацией
    model = cb.CatBoostClassifier(**param) if TASK_TYPE == 'classification' else cb.CatBoostRegressor(**param)
    
    model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        early_stopping_rounds=50,
        verbose=False
    )
    
    # Предсказания
    if TASK_TYPE == 'classification':
        preds = model.predict_proba(X_val)[:, 1]
        score = roc_auc_score(y_val, preds)
    else:
        preds = model.predict(X_val)
        score = mean_squared_error(y_val, preds, squared=False)
    
    return score

study_cb = optuna.create_study(
    direction='maximize' if TASK_TYPE == 'classification' else 'minimize',
    sampler=optuna.samplers.TPESampler(seed=42)
)

print("\nЗапуск оптимизации CatBoost...")
study_cb.optimize(objective_catboost, n_trials=30, timeout=600)

print("\n✓ Оптимизация CatBoost завершена!")
print(f"Лучший скор: {study_cb.best_value:.6f}")
print(f"Лучшие параметры: {study_cb.best_params}")

## 5. Optuna для Neural Network

In [ ]:
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X.values)
        self.y = torch.FloatTensor(y.values) if TASK_TYPE == 'regression' else torch.LongTensor(y.values)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def create_model(trial, input_dim, output_dim):
    """
    Создание нейронной сети с оптимизируемой архитектурой
    """
    n_layers = trial.suggest_int('n_layers', 1, 5)
    layers = []
    
    in_features = input_dim
    for i in range(n_layers):
        out_features = trial.suggest_int(f'n_units_l{i}', 32, 512, log=True)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        
        # Dropout
        dropout_rate = trial.suggest_float(f'dropout_l{i}', 0.0, 0.5)
        if dropout_rate > 0:
            layers.append(nn.Dropout(dropout_rate))
        
        # BatchNorm
        if trial.suggest_categorical(f'batch_norm_l{i}', [True, False]):
            layers.append(nn.BatchNorm1d(out_features))
        
        in_features = out_features
    
    # Output layer
    layers.append(nn.Linear(in_features, output_dim))
    
    if TASK_TYPE == 'classification' and output_dim == 1:
        layers.append(nn.Sigmoid())
    elif TASK_TYPE == 'classification':
        layers.append(nn.Softmax(dim=1))
    
    return nn.Sequential(*layers)

def objective_neural_network(trial):
    """
    Objective function для оптимизации Neural Network
    """
    # Гиперпараметры
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_int('batch_size', 16, 256, log=True)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'AdamW', 'SGD'])
    weight_decay = trial.suggest_float('weight_decay', 1e-8, 1e-2, log=True)
    
    # Создание модели
    input_dim = X_train.shape[1]
    output_dim = 1 if TASK_TYPE == 'regression' or len(y.unique()) == 2 else len(y.unique())
    
    model = create_model(trial, input_dim, output_dim)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Optimizer
    if optimizer_name == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'AdamW':
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=0.9)
    
    # Loss function
    if TASK_TYPE == 'classification':
        criterion = nn.BCELoss() if output_dim == 1 else nn.CrossEntropyLoss()
    else:
        criterion = nn.MSELoss()
    
    # DataLoaders
    train_dataset = TabularDataset(X_train, y_train)
    val_dataset = TabularDataset(X_val, y_val)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    # Training loop
    n_epochs = 50
    best_val_score = float('inf') if TASK_TYPE == 'regression' else 0
    
    for epoch in range(n_epochs):
        # Training
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            optimizer.zero_grad()
            outputs = model(X_batch).squeeze()
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
        
        # Validation
        model.eval()
        val_preds = []
        val_targets = []
        
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch = X_batch.to(device)
                outputs = model(X_batch).squeeze()
                val_preds.extend(outputs.cpu().numpy())
                val_targets.extend(y_batch.numpy())
        
        # Score
        if TASK_TYPE == 'classification':
            val_score = roc_auc_score(val_targets, val_preds)
            if val_score > best_val_score:
                best_val_score = val_score
        else:
            val_score = mean_squared_error(val_targets, val_preds, squared=False)
            if val_score < best_val_score:
                best_val_score = val_score
        
        # Pruning
        trial.report(best_val_score, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()
    
    return best_val_score

study_nn = optuna.create_study(
    direction='maximize' if TASK_TYPE == 'classification' else 'minimize',
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
    sampler=optuna.samplers.TPESampler(seed=42)
)

print("\nЗапуск оптимизации Neural Network...")
study_nn.optimize(objective_neural_network, n_trials=30, timeout=600)

print("\n✓ Оптимизация Neural Network завершена!")
print(f"Лучший скор: {study_nn.best_value:.6f}")
print(f"Лучшие параметры: {study_nn.best_params}")

## 6. Визуализация результатов

In [ ]:
import plotly

# Сравнение всех моделей
results = pd.DataFrame([
    ('XGBoost', study_xgb.best_value),
    ('LightGBM', study_lgb.best_value),
    ('CatBoost', study_cb.best_value),
    ('Neural Network', study_nn.best_value)
], columns=['Model', 'Best Score'])

print("\n" + "="*60)
print("РЕЗУЛЬТАТЫ ОПТИМИЗАЦИИ ВСЕХ МОДЕЛЕЙ")
print("="*60)
print(results.to_string(index=False))
print("="*60)

# Optimization history для лучшей модели
best_study = study_cb  # Замените на лучшую
fig = optuna.visualization.plot_optimization_history(best_study)
fig.show()

# Importance параметров
fig = optuna.visualization.plot_param_importances(best_study)
fig.show()

# Parallel coordinate plot
fig = optuna.visualization.plot_parallel_coordinate(best_study)
fig.show()

## 7. Multi-objective optimization

In [ ]:
def objective_multi(trial):
    """
    Multi-objective: оптимизируем accuracy И скорость предсказания
    """
    import time
    
    # Параметры модели
    param = {
        'iterations': trial.suggest_int('iterations', 50, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'random_state': 42,
        'verbose': 0
    }
    
    model = cb.CatBoostClassifier(**param) if TASK_TYPE == 'classification' else cb.CatBoostRegressor(**param)
    
    # Обучение
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=False)
    
    # Метрика качества
    if TASK_TYPE == 'classification':
        preds = model.predict_proba(X_val)[:, 1]
        accuracy = roc_auc_score(y_val, preds)
    else:
        preds = model.predict(X_val)
        accuracy = -mean_squared_error(y_val, preds, squared=False)
    
    # Скорость инференса
    start_time = time.time()
    _ = model.predict(X_val)
    inference_time = time.time() - start_time
    
    return accuracy, -inference_time  # Максимизируем оба (поэтому -time)

# Multi-objective study
study_multi = optuna.create_study(
    directions=['maximize', 'maximize'],  # Оба объектива на максимум
    sampler=optuna.samplers.NSGAIISampler(seed=42)
)

print("\nЗапуск multi-objective optimization...")
study_multi.optimize(objective_multi, n_trials=50)

print("\n✓ Multi-objective optimization завершена!")
print(f"Количество Pareto-оптимальных решений: {len(study_multi.best_trials)}")

# Визуализация Pareto front
fig = optuna.visualization.plot_pareto_front(study_multi, target_names=['Accuracy', 'Speed'])
fig.show()

## 8. Обучение финальной модели

In [ ]:
# Используем лучшие параметры для обучения на всех данных
best_params = study_cb.best_params  # Замените на лучшую модель

if TASK_TYPE == 'classification':
    best_params['loss_function'] = 'Logloss'
    best_params['eval_metric'] = 'AUC'
    final_model = cb.CatBoostClassifier(**best_params, random_state=42, verbose=0)
else:
    best_params['loss_function'] = 'RMSE'
    final_model = cb.CatBoostRegressor(**best_params, random_state=42, verbose=0)

# Обучение на всех данных
final_model.fit(X, y)

# Предсказания
if TASK_TYPE == 'classification':
    predictions = final_model.predict_proba(X_test)[:, 1]
else:
    predictions = final_model.predict(X_test)

print("\n✓ Финальная модель обучена!")

## 9. Submission

In [ ]:
submission = pd.DataFrame({
    ID_COL: test_df[ID_COL],
    'prediction': predictions
})

submission.to_csv('optuna_submission.csv', index=False)
print("\n✓ Submission сохранен!")
print(submission.head())